In [1]:
# Setup the Jupyter version of Dash
#from jupyter_plotly_dash import JupyterDash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import re


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import AnimalShelter from CRUD Python module
from CRUDMongoModule import CRUDMongo

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "password"
host = 'nv-desktop-services.apporto.com'
port = 30243
database = 'AAC'
collection = "animals"

db = CRUDMongo(username, password, host, port, database, collection)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)
# print(df)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('MyDashboard')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    # radio buttons
    dcc.RadioItems(id='rescue-radio-buttons', 
                   options=['Water Rescue', 'Mountain Rescue','Disaster Rescue', 'Reset'], value=''),
    dash_table.DataTable(id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} 
                 # Set up the features for interactive data table to make it user-friendly for client
                for i in df.columns],
                         data=df.to_dict('records'),
                         editable = False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[],
                         page_action="native",
                         page_current=0,
                         page_size=10
                         ),
    html.Br(),
    html.Hr(),
    # graph next to map
     html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
             ]),
    # Clickable Logo
    html.A([html.Img(src=app.get_asset_url('GS_Logo.png'), style={'height' : '7%', 'width' : '7%'})], href='https://www.snhu.edu'),
    # unique identifier code
    html.H4("Phoebe Johnson SNHU CS-340 MongoDB Authentication")
    
])


#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
# @app.callback(Output('datatable-id', 'style_data_conditional'),
#    [Input('datatable-id', 'selected_columns')]
# )

# queries for radio buttons
water_rescue = db.read({"$or" : [{"breed" : {"$regex" : "Labrador Retriever"}},
                                                        {"breed" : {"$regex" : "Chesa Bay"}},
                                                        {"breed" : {"$regex" : "Newfoundland"}}],
                                                "sex_upon_outcome" : "Intact Female",
                                                "age_upon_outcome_in_weeks" : {"$gte" : 26, "$lte" : 156}
                                                })

mountain_rescue = db.read({"$or" : [{"breed" : {"$regex" : "German Shepherd"}},
                                                        {"breed" : {"$regex" : "Alaskan Malamute"}},
                                                        {"breed" : {"$regex" : "Old English Sheepdog"}},
                                                        {"breed" : {"$regex" : "Siberian Husky"}},
                                                        {"breed" : {"$regex" : "Rottweiler"}}],
                                                "sex_upon_outcome" : "Intact Male",
                                                "age_upon_outcome_in_weeks" : {"$gte" : 26, "$lte" : 156}
                                                })

disaster_rescue = db.read({"$or" : [{"breed" : {"$regex" : "Doberman Pinsch"}},
                                                        {"breed" : {"$regex" : "German Shepherd"}},
                                                        {"breed" : {"$regex" : "Golden Retriever"}},
                                                        {"breed" : {"$regex" : "Bloodhound"}},
                                                        {"breed" : {"$regex" : "Rottweiler"}}],
                                                "sex_upon_outcome" : "Intact Male",
                                                "age_upon_outcome_in_weeks" : {"$gte" : 20, "$lte" : 300}
                                                })

# callback for radio buttons
@app.callback(
    Output('datatable-id', 'data'),
    Input('rescue-radio-buttons', 'value')
)
def update_chart(choice):

    if choice == 'Water Rescue':
        df = pd.DataFrame.from_records(water_rescue)
    elif choice == 'Mountain Rescue':
        df = pd.DataFrame.from_records(mountain_rescue)
    elif choice == 'Disaster Rescue':
        df = pd.DataFrame.from_records(disaster_rescue)
    else:
        # default choice and reset choice
        df = pd.DataFrame.from_records(db.read({}))
        
        # Cleanup Mongo_id field
    df.drop(columns=['_id'], inplace=True)
    
    return df.to_dict('records')

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# callback for histogram
@app.callback(Output('graph-id', "children"),
             [Input('datatable-id', 'derived_viewport_data'),
              Input('rescue-radio-buttons', 'value')
             ])
def update_graph(viewData, choice):
    dff=pd.DataFrame.from_dict(viewData)
    
    return[
        dcc.Graph(
            figure = px.histogram(dff, x=['breed'])
        )
    ]

# callback for map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")
    ])
def update_map(viewData, index):
# Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can
    # be converted to a row index here
    if index is None or not index:
        row = 0
    else:
        row = index[0]
        
        # Austin TX is at [30.75, -97.48]
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
              center=[30.75, -97.48], 
               zoom=10, 
               children=[
                  dl.TileLayer(id="base-layer-id"),
                  # Marker with tool tip and popup
                  # Column 13 and 14 define the grid-coordinates for
                  # the map
                  # Column 4 defines the breed for the animal
                  # Column 9 defines the name of the animal
                  dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                        children=[
                        dl.Tooltip(dff.iloc[row,4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row,9])
                        ])
                    ])
                ])
            ]
    

app.run_server(debug=True)

Dash app running on http://127.0.0.1:15744/
